In [38]:
import os
# Import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import CohereEmbeddings
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
index_name = "langchain-test-index-3"

# index_name = "cohere-test-embeddings"
existing_indexes = [
    index["name"] for index in pc.list_indexes()
]
index_exists = index_name in existing_indexes

# Create an Index
if not index_exists:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

embeddings_model = CohereEmbeddings(
    model="embed-english-light-v3.0",
    cohere_api_key=os.environ.get("COHERE_API_KEY"),
)

vectorstore_a = PineconeVectorStore(
    embedding=embeddings_model,
    index_name=index_name
)

In [39]:
vectorstore_a.add_texts(["USC is the best school in the world", 
                         "USC is known for being awesome because it is very warm during the summer",
                        "USC has the best college of Letter's of Arts and Sciences called Dornsife Viterbi",
                         "USC has the best dance school, Kauffman, better than any other dance school in the world (Juliard, the person)",
                         "USC is known for being bad because it has many hard linguistics and CS classes",
                         "USC is bad because Complex Analysis is too Complex, and Accounting is not accounted for"]
                        )

['ac2302b1-7a49-4245-8cae-23e54dbab2cf',
 '3c9465e5-dbab-4fdb-b12d-56f3c9ad17cf',
 'c273ae3c-f272-4172-ba9c-8ee9f5a18417',
 '27c9965c-23d3-461b-9f30-6142c8ae0dc2',
 '4bf41eb4-a3f5-4bec-86a4-0a46f45ca654',
 '41bdab22-461e-4127-bd7b-20d3d3a3d07f']

In [40]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.llms import Cohere

llm = Cohere(model="command")

retriever = vectorstore_a.as_retriever()
prompt_str = """
Question: {question}

Please answer the question above using only the context provided.
Context: {context}

Answer:
"""

prompt =  PromptTemplate.from_template(prompt_str)

retrieve_context_and_get_quality = RunnableParallel({"question": RunnablePassthrough(), "context": retriever})

chain = retrieve_context_and_get_quality | prompt | llm  | StrOutputParser()

In [45]:
out = chain.invoke("Why is USC bad?")
print(out)

 Some reasons why some people think USC is bad are because of the difficulty of its linguistics and CS classes, and how Complex Analysis is complex and that accounting is not accounted for.


In [46]:
out = chain.invoke("Why is USC good?")
print(out)

 USC is known for its intense and difficult courses in linguistics and CS, but also for its incredibly warm summer. It is a highly acclaimed institution for higher education, world-renowned as the best school in the world. 

Is there any more information I can provide about the University of Southern California (USC)? 


In [47]:
out = chain.invoke("Tell me about USC's Dance Program?")
print(out)

 USC's dance program is the best in the world and is housed in the equally acclaimed Kauffman school of dance. 

An unnamed source remarks that it is better than the Juliard [sic] school. 

It is part of the College of Letters of Arts and Sciences called Dornsife Viterbi which highlights the liberal arts and sciences component of the University of Southern California. 

So, in summation, USC's dance program, administered through the Kauffman School of Dance, is the absolute best in the world and emphasizes a liberal arts and science education with its Letter's of Arts and Sciences college. 

Is there anything else you would like to know about USC's dance program or any of the other programs affiliated with the university? 


In [44]:
# Cleanup
# pc.delete_index(name=index_name)